In [7]:
# auto_mbe3_control.py
# -*- coding: utf-8 -*-

import pyvisa
import time
import datetime
import sys
import serial
import re
import json
import os
import matplotlib.pyplot as plt
import pandas as pd

# ----------------------- Utility Functions -----------------------
def format_duration(sec):
    if sec < 3600:
        return f"{sec/60:.2f} min"
    else:
        return f"{sec/3600:.2f} hr"

def show_progress(elapsed, total, length=30):
    frac   = min(elapsed/total, 1)
    filled = int(frac*length)
    bar    = "#"*filled + "-"*(length-filled)
    return f"[{bar}] {frac*100:5.1f}% ({format_duration(elapsed)}/{format_duration(total)})"

# ----------------------- Record Total Start Time -----------------------
total_start = time.time()

# ----------------------- User Input & Folder Setup -----------------------
sample_id = input("Enter sample ID (e.g. S1, A03): ").strip()
date_str  = datetime.datetime.now().strftime("%Y%m%d")
folder    = f"{date_str}_{sample_id}"
os.makedirs(folder, exist_ok=True)

# ----------------------- Load Config & Save Params -----------------------
with open('params.json','r') as f:
    config = json.load(f)
with open(os.path.join(folder, f"{date_str}_{sample_id}_params.txt"), 'w') as f:
    for k,v in config.items():
        f.write(f"{k} = {v}\n")

# ----------------------- Extract Config Values -----------------------
dt1           = config["dt1"]
dI1           = config["dI1"]
Imax          = config["Imax"]
I             = config["I"]
T1            = config["T1"]
degas_time1   = config["degas_time1_hr"] * 3600

dt2           = config["dt2"]
dI2           = config["dI2"]
T2            = config["T2"]
degas_time2   = config["degas_time2_min"] * 60

T_mid         = config["T_mid"]
dt3a          = config["dt3a"]
dI3a          = config["dI3a"]
dt3b          = config["dt3b"]
dI3b          = config["dI3b"]

T3            = config["T3"]
anneal_time   = config["anneal_time_min"] * 60

cooling_time1 = config["cooling_time1_min"] * 60
cooling_drop  = config["cooling_drop"]
cooling_time2 = config["cooling_time2_min"] * 60
cooling_target= config["cooling_target"]

# ----------------------- Initialize Instruments -----------------------
rm = pyvisa.ResourceManager()
power_supply = rm.open_resource('USB0::0x2EC7::0x6900::800778011777310077::INSTR')
power_supply.timeout = 5000
# Turn on the output once
power_supply.write('OUTPut:STATe ON')

# ----------------------- Initialize Pyrometer -----------------------
ser = serial.Serial(
    port='COM5', baudrate=19200,
    bytesize=serial.EIGHTBITS, stopbits=serial.STOPBITS_ONE,
    parity=serial.PARITY_EVEN, timeout=1
)

# ----------------------- Data Logging -----------------------
log_records = []
def write_log(phase, Vm, Im, T):
    log_records.append({
        'timestamp': datetime.datetime.now().isoformat(),
        'phase': phase,
        'voltage_V': round(Vm,3),
        'current_A': round(Im,3),
        'temperature_C': round(T if T is not None else -1,1)
    })

# ----------------------- Helper: Read Temperature -----------------------
temp_fail = 0
def read_temperature():
    global temp_fail
    ser.write(b'00ms\r')
    time.sleep(1)
    buf = b''
    while ser.inWaiting() > 0:
        buf += ser.read(10)
    nums = re.findall(rb'\d+', buf)
    if nums:
        temp_fail = 0
        return float(nums[0]) / 10.0
    temp_fail += 1
    if temp_fail > 10:
        ser.close()
        sys.exit("ERROR: Too many temperature read failures")
    return None

# ----------------------- Phase Functions -----------------------
def adjust_and_hold(target, dI, dt, Imax, I, duration, phase):
    print(f"\n--- {phase}: ramp to ≥{target}°C ---")
    while True:
        T = read_temperature()
        if T is None: continue
        if T >= target or I >= Imax:
            print(f"{phase} reached: T={T:.1f}°C – start degas")
            break
        I += dI
        power_supply.write(f"SOUR:CURR {I:.4f}")
        time.sleep(dt)
        Vm = float(power_supply.query("MEAS:VOLT?"))
        Im = float(power_supply.query("MEAS:CURR?"))
        write_log(phase+"_ramp", Vm, Im, T)
        print(f"T={T:.1f}°C | V={Vm:.2f}V | I={Im:.2f}A")
    print(f"--- {phase}: hold at {target}°C for {format_duration(duration)} ---")
    start = time.time()
    while True:
        elapsed = time.time() - start
        if elapsed >= duration: break
        T = read_temperature()
        if T is None: continue
        if T < target-3:
            I += dI
        elif T > target+3:
            I = max(0, I-dI)
        power_supply.write(f"SOUR:CURR {I:.4f}")
        time.sleep(dt)
        Vm = float(power_supply.query("MEAS:VOLT?"))
        Im = float(power_supply.query("MEAS:CURR?"))
        write_log(phase+"_hold", Vm, Im, T)
        print(f"T={T:.1f}°C | V={Vm:.2f}V | I={Im:.2f}A | {show_progress(elapsed,duration)}")
    print(f"{phase} completed")
    return I

def adjust_temperature(target, dI, dt, Imax, I, phase):
    print(f"\n--- {phase}: ramp to ≥{target}°C ---")
    while True:
        T = read_temperature()
        if T is None: continue
        if T >= target or I >= Imax:
            print(f"{phase} reached: T={T:.1f}°C")
            break
        I += dI
        power_supply.write(f"SOUR:CURR {I:.4f}")
        time.sleep(dt)
        Vm = float(power_supply.query("MEAS:VOLT?"))
        Im = float(power_supply.query("MEAS:CURR?"))
        write_log(phase, Vm, Im, T)
        print(f"T={T:.1f}°C | V={Vm:.2f}V | I={Im:.2f}A")
    return I

def hold_constant(I, duration, phase):
    print(f"\n--- {phase}: constant I={I:.2f}A for {format_duration(duration)} ---")
    power_supply.write(f"SOUR:CURR {I:.4f}")
    start = time.time()
    while True:
        elapsed = time.time() - start
        if elapsed >= duration: break
        T = read_temperature()
        if T is None: continue
        Vm = float(power_supply.query("MEAS:VOLT?"))
        Im = float(power_supply.query("MEAS:CURR?"))
        write_log(phase, Vm, Im, T)
        print(f"T={T:.1f}°C | V={Vm:.2f}V | I={Im:.2f}A | {show_progress(elapsed,duration)}")
        time.sleep(5)
    print(f"{phase} completed")
    return I

def ramp_down_time_based(I_start, drop, total_seconds, phase):
    print(f"\n--- {phase}: cooling down for {format_duration(total_seconds)} ---")
    t0 = time.time()
    end = t0 + total_seconds
    while True:
        now = time.time()
        if now >= end: break
        elapsed = now - t0
        I_set = I_start - drop * (elapsed/total_seconds)
        power_supply.write(f"SOUR:CURR {I_set:.4f}")
        T = read_temperature()
        Vm = float(power_supply.query("MEAS:VOLT?"))
        Im = float(power_supply.query("MEAS:CURR?"))
        write_log(phase, Vm, Im, T)
        print(f"T={T:.1f}°C | V={Vm:.2f}V | I={Im:.2f}A | {show_progress(elapsed,total_seconds)}")
        time.sleep(5)
    final_I = I_start - drop
    power_supply.write(f"SOUR:CURR {final_I:.4f}")
    print(f"{phase} completed")
    return final_I

# ----------------------- Execution Sequence -----------------------
I = adjust_and_hold(T1,   dI1, dt1, Imax, I,    degas_time1, "Phase 1")
phase1_end = datetime.datetime.now()

I = adjust_temperature(T2, dI2, dt2, Imax, I, "Phase 2")
I = hold_constant    (I,   degas_time2,       "Phase 2")

# two-stage ramp
I = adjust_temperature(T_mid, dI3a, dt3a, Imax, I, "Phase 3a")
I = adjust_temperature(T3,    dI3b, dt3b, Imax, I, "Phase 3b")

I = hold_constant    (I,   anneal_time,       "Phase 3")

I = ramp_down_time_based(I, cooling_drop,     cooling_time1, "Cooling 1")
I = ramp_down_time_based(I, I-cooling_target, cooling_time2, "Cooling 2")

# ----------------------- Save & Plot -----------------------
df = pd.DataFrame(log_records)
df['timestamp'] = pd.to_datetime(df['timestamp'])

all_csv = os.path.join(folder, f"{date_str}_{sample_id}_all.csv")
df[['timestamp','temperature_C','voltage_V','current_A']].to_csv(all_csv, index=False)

df_after = df[df['timestamp'] >= phase1_end]

def plot_and_save(df_, suffix):
    times = df_['timestamp']
    for col,ylabel in [
        ('temperature_C','Temperature (°C)'),
        ('voltage_V','Voltage (V)'),
        ('current_A','Current (A)')
    ]:
        plt.figure(figsize=(10,6))
        plt.plot(times, df_[col], '-o', markersize=4)
        plt.xlabel("Time"); plt.ylabel(ylabel)
        plt.title(f"{ylabel} vs Time ({suffix})")
        plt.grid(True); plt.tight_layout()
        plt.savefig(os.path.join(folder, f"{date_str}_{sample_id}_{col}_{suffix}.png"))
        plt.close()

plot_and_save(df,       'full')
plot_and_save(df_after, 'postphase1')

total_elapsed = time.time() - total_start
print(f"\n✅ All done. Total runtime: {format_duration(total_elapsed)}")
print(f"Data & plots saved in: {all_csv}")


Enter sample ID (e.g. S1, A03):  SiC#18 5 layer C60 4mins K



--- Phase 1: ramp to ≥550°C ---
T=149.0°C | V=1.35V | I=0.01A
T=149.0°C | V=3.88V | I=0.02A
T=149.0°C | V=4.77V | I=0.03A
T=149.0°C | V=5.48V | I=0.04A
T=149.0°C | V=6.03V | I=0.05A
T=149.0°C | V=6.52V | I=0.06A
T=149.0°C | V=6.98V | I=0.07A
T=149.0°C | V=7.40V | I=0.08A
T=149.0°C | V=7.78V | I=0.09A
T=149.0°C | V=8.10V | I=0.10A
T=149.0°C | V=8.40V | I=0.11A
T=149.0°C | V=8.56V | I=0.12A
T=149.0°C | V=8.85V | I=0.13A
T=149.0°C | V=9.12V | I=0.14A
T=149.0°C | V=9.39V | I=0.15A
T=149.0°C | V=9.63V | I=0.16A
T=149.0°C | V=9.85V | I=0.17A
T=149.0°C | V=10.05V | I=0.18A
T=149.0°C | V=10.22V | I=0.19A
T=149.0°C | V=10.46V | I=0.20A
T=149.0°C | V=10.61V | I=0.21A
T=149.0°C | V=10.65V | I=0.22A
T=149.0°C | V=10.72V | I=0.23A
T=156.3°C | V=10.76V | I=0.24A
T=165.7°C | V=10.72V | I=0.25A
T=174.6°C | V=10.65V | I=0.26A
T=183.5°C | V=10.62V | I=0.27A
T=191.8°C | V=10.61V | I=0.28A
T=200.3°C | V=10.52V | I=0.29A
T=208.4°C | V=10.47V | I=0.30A
T=216.7°C | V=10.40V | I=0.31A
T=225.0°C | V=10.34V | 

In [8]:
import pyvisa as visa
import time
import datetime
import sys
import serial
import re
import threading

# Establish connection using VISA
rm = visa.ResourceManager()

# Replace this with the correct VISA address for your instrument
power_supply = rm.open_resource('USB0::0x2EC7::0x6900::800778011777310077::INSTR')  # Example VISA address



# Function to gradually change the current and wait for the specified time gap
def gradually_change_current(I_start, I_end, time_gap_minutes, steps=100):
    """
    Gradually change current from I_start to I_end over time_gap_minutes.
    The time_gap is given in minutes, and current is incremented over small steps.
    """
    # Convert the time gap from minutes to seconds
    time_gap_seconds = time_gap_minutes * 60
    
    # Calculate the step size for current (increment per step)
    current_step = (I_end - I_start) / steps
    
    # Calculate the time for each step (increment)
    step_time = time_gap_seconds / steps
    
    # Start time to track elapsed time
    start_time = time.time()
    
    # Gradually update the current in small steps
    for step in range(steps):
        # Calculate the new current value for this step
        current_value = I_start + current_step * (step + 1)
        
        # Set the current to the new value using the SCPI command for ITECH
        power_supply.write(f'SOURce:CURRent:LEVel {current_value:.4f}')  # Set current
        
        # Turn on the output (only once at the start)
        if step == 0:
            power_supply.write('OUTPut:STATe ON')  # Turn on output for ITECH IT6942A
        
        # Read the voltage and current from the power supply

        if step % 5 == 0:
            power_supply.write('MEAS:VOLT?')  # Query voltage
            voltage = power_supply.read('')
            power_supply.write('MEAS:CURR?')  # Query voltage
            current = power_supply.read('')
            
            # Read the elapsed time
            elapsed_time = time.time() - start_time
            
            # Print real-time data
            #print(f"Step {step + 1}/{steps}: Voltage = {voltage.strip()} V, Current = {current.strip()} A, Elapsed time = {elapsed_time:.2f} s")
            print(f"\rStep {step + 1}/{steps}: Voltage = {voltage.strip()} V, Current = {current.strip()} A, Elapsed time = {elapsed_time:.2f} s", end='', flush=True)
            
        # Wait for the next step
        time.sleep(step_time)
    
    print(f"Gradual change from {I_start} A to {I_end} A completed in {time_gap_minutes} minutes.")

# Function to handle the sequence of current changes
def change_currents(I, t_minutes):
    # Check that the lengths of I and t match
    if len(I) != len(t_minutes):
        print("Error: Length of currents list I must match the length of time list t.")
        return
    
    # Loop through each current and its corresponding time gap
    for i in range(1, len(I)):
        # Calculate the time gap (tn - t(n-1))
        time_gap = t_minutes[i] - t_minutes[i-1]
        
        # Gradually change the current from I[i-1] to I[i] over the time gap
        gradually_change_current(I[i-1], I[i], time_gap)
    
    print("All current steps completed.")

# Example current values (I) in Amps and corresponding times (t) in minutes
I = [0.45,0.45,0.0] # Current values in Amps (I1, I2, ..., In)
t_minutes = [0,5,10]  # Times in minutes (t1, t2, ..., tn)


# Change currents based on the lists I and t_minutes
change_currents(I, t_minutes)


# Keep the output ON at the final current (no turning off)
print("Final output is at current:", I[-1], "A")


Step 96/100: Voltage = 2.193 V, Current = 0.4494 A, Elapsed time = 285.40 sGradual change from 0.45 A to 0.45 A completed in 5 minutes.
Step 96/100: Voltage = 0.13 V, Current = 0.022 A, Elapsed time = 285.40 s sGradual change from 0.45 A to 0.0 A completed in 5 minutes.
All current steps completed.
Final output is at current: 0.0 A


In [ ]:
import pyvisa as visa
import time
import datetime
import sys
import serial
import re
import threading

rm = visa.ResourceManager()

# Replace this with your correct VISA address
power_supply = rm.open_resource('USB0::0x2EC7::0x6900::800778011777310077::INSTR')  # Example VISA address
power_supply.write(f'SOURce:VOLTage:LEVel {40.0}')  # Set voltage
#power_supply.write(f'SOURce:CURRent:LEVel {0.0}')  # Set current